**Vanessa Navarro Coronado e Iván Sánchez Castellanos**

# Práctica 2 - Clasificación supervisada en scikit-learn

En esta práctica vamos a estudiar de nuevo los datasets Pima y Wisconsin. Probaremos los clasificadores de Árbol de Decisión y KNN, y analizaremos cuál es mejor en cada caso. Además, trataremos de descubrir las configuraciones óptimas de dichos algoritmos, y realizaremos un breve estudio sobre ellos.

También realizaremos la parte opcional, en la aplicaremos los mismos procedimientos que usamos con el árbol de decisión y el KNN, utilizando un RandomizedSearch en lugar del GridSearch para conseguir la mejor configuración de parámetros en nuestros clasificadores.

Para comenzar, como siempre, lo primero que hacemos es importar los paquetes necesarios, e inicializar nuestra semilla para nuestros experimentos.

In [1]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [2]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [3]:
seed=6342
np.random.seed(6342)

### Cargado de los datos

A continuación, cargamos los datos de los datasets Pima y Wisconsin. Cada uno será separado en dos conjuntos, uno con los atributos y otro con la clase del dataset. Esto es necesario para que la libreria scikit learn pueda ser utilizada.

In [4]:
# Update the file path to fit your system
dfPima = pd.read_csv("../data/pima.csv", dtype={ "label": 'category'})
dfAttributesPima = dfPima.drop('label', 1)
dfLabelPima = dfPima['label']

dfWisc = pd.read_csv("../data/wisconsin.csv", dtype={ "label": 'category'})
dfAttributesWisc = dfWisc.drop('label', 1)
dfLabelWisc = dfWisc['label']

Una vez cargados los datasets, debemos dividirlos en dos conjuntos de Train y Test para poder realizar nuestros experimentos.

In [5]:
# Divide into train/test split for our experiments
from sklearn.model_selection import train_test_split
train_attsPima, test_attsPima, train_labelPima, test_labelPima = train_test_split( 
    dfAttributesPima,
    dfLabelPima,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelPima)

train_attsWisc, test_attsWisc, train_labelWisc, test_labelWisc = train_test_split( 
    dfAttributesWisc,
    dfLabelWisc,
    test_size=0.2,
    random_state=seed,
    stratify=dfLabelWisc)

### Preprocesamiento de los datos en dataset Pima

Para poder realizar el tratamiento de valores perdidos en el dataset Pima, primero debemos seleccionar los atributos que contengan valores a cero que son considerados como perdidos. En este caso nosotros pensamos que un cero en la variable 'preg' no sería valor perdido, ya que los pacientes puede que no hayan estado embarazados ninguna vez, o que sean de género masculino. Por tanto, un cero en cualquiera de las otras variables predictoras será considerado como valor perdido, a excepción de la variable 'preg'.

In [6]:
aux_ceros = dfPima.columns.drop(["preg","label"])
aux_ceros

Index(['plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')

Una vez separadas las variables sobre las que queremos hacer tratamiento de valores perdidos, reemplazamos los valores a cero por valores de tipo NaN, para que podamos imputar estos valores por la media de las variables para nuestro dataset.

In [7]:
for i in aux_ceros:
    train_attsPima.replace({i: {0: np.nan}}, inplace = True)
    train_labelPima.replace({i: {0: np.nan}}, inplace = True)
    test_attsPima.replace({i: {0: np.nan}}, inplace = True)
    test_labelPima.replace({i: {0: np.nan}}, inplace = True)

### Preprocesamiento de los datos en dataset Wisconsin

Para el dataset Wisconsin, solamente prescindiremos de la variable 'patientId', como ya hicimos en la práctica anterior, porque esta variable es irrelevante.

In [8]:
train_attsWisc2=train_attsWisc.drop('patientId',1)
test_attsWisc2=test_attsWisc.drop('patientId',1)

# 1. Selección y evaluación de modelos

En esta sección realizaremos experimentos utilizando el algoritmo GridSearch de la librería scikit-learn para descubrir las configuraciones óptimas de los clasificadores DecisionTree y KNN para los datasets Pima y Wisconsin.

In [9]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

Los pasos a seguir en este apartado son:
* Crear un Pipeline con dos pasos:
    - Un objeto de la clase Imputer que se encarga de realizar el tratamiento de valores perdidos. En nuestro caso, imputaremos los valores NaN de los datasets con la media.
    - Un clasificador como estimador (árbol de decisión o KNN en nuestro caso).
* Crear un objeto de la clase GridSearchCV, al que le pasaremos:
    - el estimador que usamos en el Pipeline, 
    - los hiperparámetros del clasificador que queremos tener en cuenta a la hora de buscar la configuración óptima.
    - También debemos indicar qué tipo de métrica usaremos para valorar las configuraciones,
    - cuántos folds se deben realizar en el proceso de validación cruzada que realiza el GridSearchCV. Si en este campo especificamos un objeto de la clase StratifiedKFold con 10 folds y nuestra semilla, podrá realizar un proceso de validación cruzada estratificada.
    - iid = False, para que la evaluación de los resultados se haga sobre una media aritmética, y no sobre una media ponderada.
* Entrenar y validar el clasificador con los datos de nuestros datasets, y analizar los resultados obtenidos (accuracy, precision y recall).

## 1.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

Comenzamos con el clasificador DecisionTree.

In [10]:
from sklearn.preprocessing import Imputer
# Cargamos el arbol de decision
from sklearn import tree

from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

### 1.1.1 Pipeline

A continuación creamos un Pipeline con dos pasos: el primero es imputar los valores perdidos siguiendo la estrategia de rellenar con la media, y el segundo paso es un clasificador DecisionTree, con su semilla para poder hacer reproducibles nuestros experimentos.

In [11]:
from sklearn.pipeline import Pipeline

In [12]:
# We define the pipeline as a set of tuples
estimatorTree = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("Tree", tree.DecisionTreeClassifier(random_state=seed))])

### 1.1.2. GridSearch

Aquí definimos nuestro GridSearch con los parámetros elegidos. Como estimador debemos pasarle el que usamos en nuestro Pipeline.

En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [13]:
clfTree = GridSearchCV(
    estimator = estimatorTree,
    param_grid =
        {'Tree__criterion': ["entropy","gini"],'Tree__max_depth': [3, 5, 10, None], 'Tree__min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), iid=False
)

Si asignamos el valor 'None' al parámetro 'max_depth', el clasificador intentará llegar a la máxima profundidad del árbol por defecto.

### 1.1.3. Pima

Después de pasar el GridSearch, ejecutamos con el dataset Pima para entrenar el clasificador con el conjunto de Train, validamos con nuestro conjunto de Test, y después imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro GridSearch.

In [14]:
# We can fit and use the pipeline as usual
clsTreePima = clfTree.fit(train_attsPima, train_labelPima)
predictionTreePima = clfTree.predict(test_attsPima)
print('Accuracy Pima Tree:')
metrics.accuracy_score(test_labelPima, predictionTreePima)

Accuracy Pima Tree:


0.72727272727272729

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado el GridSearch. En este caso, la mejor configuración es utilizar el criterio de entropía para evaluar las variables del árbol, emplear una profundidad máxima del árbol de 10, y un número mínimo de ejemplos por hoja igual a 10.

In [15]:
print('Mejor configuración de parámetros (Pima Tree):')
clsTreePima.best_params_

Mejor configuración de parámetros (Pima Tree):


{'Tree__criterion': 'entropy',
 'Tree__max_depth': 10,
 'Tree__min_samples_leaf': 10}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida, y las medidas de precision y recall.

In [16]:
print('Confusion matrix Pima Tree:')
metrics.confusion_matrix(test_labelPima, predictionTreePima)

Confusion matrix Pima Tree:


array([[79, 21],
       [21, 33]])

In [17]:
# Recall
print('Recall Pima Tree:')
metrics.recall_score(test_labelPima, predictionTreePima, pos_label="tested_positive")

Recall Pima Tree:


0.61111111111111116

In [18]:
# Precision
print('Precision Pima Tree:')
metrics.precision_score(test_labelPima, predictionTreePima, pos_label="tested_positive")

Precision Pima Tree:


0.61111111111111116

### 1.1.4. Wisconsin

Ahora realizamos el mismo procedimiento con el dataset Wisconsin. Usando el Pipeline creado en el apartado 1.1.1., entrenamos, validamos y obtenemos las métricas.

In [19]:
# We can fit and use the pipeline as usual
clsTreeWisc = clfTree.fit(train_attsWisc2, train_labelWisc)
predictionTreeWisc = clfTree.predict(test_attsWisc2)
print('Accuracy Wisconsin Tree:')
metrics.accuracy_score(test_labelWisc, predictionTreeWisc)


Accuracy Wisconsin Tree:


0.95714285714285718

Ahora mostramos la mejor configuración obtenida con el GridSearch, que en este caso consiste en usar el criterio gini para el árbol, una profundidad máxima de 5, y un número mínimo de ejemplos por hoja igual a 5.

In [20]:
print('Mejor configuración de parámetros (Wisconsin Tree):')
clsTreeWisc.best_params_

Mejor configuración de parámetros (Wisconsin Tree):


{'Tree__criterion': 'gini', 'Tree__max_depth': 5, 'Tree__min_samples_leaf': 5}

A continuación, mostramos la matriz de confusión, precision y recall, igual que hicimos con el dataset Pima.

In [21]:
print('Confusion matrix Wisconsin Tree:')
metrics.confusion_matrix(test_labelWisc, predictionTreeWisc)

Confusion matrix Wisconsin Tree:


array([[90,  2],
       [ 4, 44]])

In [22]:
# Recall
print('Recall Wisconsin Tree:')
metrics.recall_score(test_labelWisc, predictionTreeWisc, pos_label="malignant")

Recall Wisconsin Tree:


0.91666666666666663

In [23]:
# Precision
print('Precision Wisconsin Tree:')
metrics.precision_score(test_labelWisc, predictionTreeWisc, pos_label="malignant")

Precision Wisconsin Tree:


0.95652173913043481

## 1.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

En esta parte vamos a usar un KNN como clasificador para nuestro Pipeline.

In [24]:
from sklearn import neighbors

In [25]:
import sklearn.metrics as metrics

In [26]:
from sklearn.model_selection import GridSearchCV

### 1.2.2. Pipeline

Creamos el Pipeline con el imputer como primer paso, y como segundo paso, el clasificador KNN.

In [27]:
# We define the pipeline as a set of tuples
estimatorKNN = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("KNN", neighbors.KNeighborsClassifier())])

### 1.2.1. GridSearch

Primero definimos los parámetros del algoritmo de KNN que queremos que el GridSearch tenga en cuenta a la hora de buscar la configuración óptima. En el algoritmo KNeighborsClassifier no se nos permite elegir un random_state como en el caso del árbol para poder hacer reproducibles nuestros experimentos, por lo que solo usaremos la semilla en el proceso de validación cruzada que se realiza en el GridSearch.

En nuestro caso, para el KNN hemos añadido el número de vecinos, y el tipo de métrica a utilizar para valorar las distancias de los vecinos (puede ser la distancia o la inversa de la distancia).

In [28]:
clfKNN = GridSearchCV(
    estimator = estimatorKNN,
    param_grid = 
        { 'KNN__n_neighbors' : [1,2,3,4,5], 'KNN__weights': ['uniform','distance'] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), iid=False
)

### 1.2.3. Pima

Primero entrenamos, validamos y obtenemos el accuracy en los datos del dataset Pima.

In [29]:
# We can fit and use the pipeline as usual
clsKNNPima = clfKNN.fit(train_attsPima, train_labelPima)
predictionKNNPima = clfKNN.predict(test_attsPima)
print('Accuracy Pima KNN:')
metrics.accuracy_score(test_labelPima, predictionKNNPima)

Accuracy Pima KNN:


0.68831168831168832

Comprobamos cuál ha sido la mejor configuración de parámetros obtenida por el GridSearch. En este caso es utilizar 5 vecinos, usando la métrica uniforme para medir las distancias.

In [30]:
print('Mejor configuración de parámetros (Pima KNN):')
clsKNNPima.best_params_

Mejor configuración de parámetros (Pima KNN):


{'KNN__n_neighbors': 5, 'KNN__weights': 'uniform'}

A continuación mostramos la matriz de confusión, precision y recall obtenidos.

In [31]:
print('Confusion matrix Pima KNN:')
metrics.confusion_matrix(test_labelPima, predictionKNNPima)

Confusion matrix Pima KNN:


array([[82, 18],
       [30, 24]])

In [32]:
# Recall
print('Recall Pima KNN:')
metrics.recall_score(test_labelPima, predictionKNNPima, pos_label="tested_positive")

Recall Pima KNN:


0.44444444444444442

In [33]:
# Precision
print('Precision Pima KNN:')
metrics.precision_score(test_labelPima, predictionKNNPima, pos_label="tested_positive")

Precision Pima KNN:


0.5714285714285714

### 1.2.4. Wisconsin

Con los datos del dataset Wisconsin, entrenamos nuestro clasificador, validamos y obtenemos el accuracy.

In [34]:
# We can fit and use the pipeline as usual
clsKNNWisc = clfKNN.fit(train_attsWisc2, train_labelWisc)
predictionKNNWisc = clfKNN.predict(test_attsWisc2)
print('Accuracy Wisconsin KNN:')
metrics.accuracy_score(test_labelWisc, predictionKNNWisc)

Accuracy Wisconsin KNN:


0.97857142857142854

Los mejores parámetros obtenidos por el GridSearch para el KNN en este dataset son utilizar 3 vecinos, usando la métrica uniforme para medir las distancias.

In [35]:
print('Mejor configuración de parámetros (Wisconsin KNN):')
clsKNNWisc.best_params_

Mejor configuración de parámetros (Wisconsin KNN):


{'KNN__n_neighbors': 3, 'KNN__weights': 'uniform'}

A continuación, mostramos la matriz de confusión, precision y recall obtenidos.

In [36]:
print('Confusion matrix Wisconsin KNN:')
metrics.confusion_matrix(test_labelWisc, predictionKNNWisc)

Confusion matrix Wisconsin KNN:


array([[91,  1],
       [ 2, 46]])

In [37]:
# Recall
print('Recall Wisconsin KNN:')
metrics.recall_score(test_labelWisc, predictionKNNWisc, pos_label="malignant")

Recall Wisconsin KNN:


0.95833333333333337

In [38]:
# Precision
print('Precision Wisconsin KNN:')
metrics.precision_score(test_labelWisc, predictionKNNWisc, pos_label="malignant")

Precision Wisconsin KNN:


0.97872340425531912

## 1.3. Comparativa

En este apartado intentaremos analizar cuál de los clasificadores ha obtenido mejores resultados en los datasets estudiados.

### 1.3.1. Pima

En el dataset Pima los resultados de accuracy para el árbol de decisión y el KNN con k=5, son: 
* Accuracy Pima Tree: 0.72727272727272729
* Accuracy Pima 5-NN: 0.68831168831168832

En cuanto a los resultados de recall tenemos:
* Recall Pima Tree: 0.61111111111111116
* Recall Pima 5-NN: 0.44444444444444442

Y los de precision:
* Precision Pima Tree: 0.61111111111111116
* Precision Pima 5-NN: 0.5714285714285714

Por tanto, podemos concluir que el mejor modelo es el obtenido con el árbol de decisión.

### 1.3.2. Wisconsin

En el dataset Wisconsin, los resultados de acurracy para el árbol de decisión y el KNN con k=3 son:
* Accuracy Wisconsin Tree: 0.95714285714285718
* Accuracy Wisconsin 3-NN: 0.97857142857142854

Los resultados de recall son:
* Recall Wisconsin Tree: 0.91666666666666663
* Recall Wisconsin 3-NN: 0.95833333333333337

Y los de precision:
* Precision Wisconsin Tree: 0.95652173913043481
* Precision Wisconsin 3-NN: 0.97872340425531912

Como vemos, el mejor modelo para este dataset podría ser el KNN en este caso.

## 1.4. Estudio de los algoritmos

En este apartado vamos a estudiar invidualmente los algoritmos:
* En KNN, estudiaremos los parámetros aprendidos por el clasificador, realizando varias pruebas con distintas configuraciones de parámetros y analizando los resultados. Analizaremos por separado los dos datasets.
* Para el árbol, estudiaremos los parámetros aprendidos por el clasificador, y también su estructura. Analizaremos las estructuras del clasificador con la configuración óptima y otra con una configuración suboptima, y valoraremos los resultados.

Para analizarlos, primero vamos a crear una funcion getScores que será similar a nuestra implementación del GridSearch pero más sencilla. Simplemente obtendremos los scores usando los distintos parámetros del clasificador. No realizamos proceso de validación cruzada, solamente usamos el holdout que teníamos al principio de la práctica.

Después obtenemos tiempo y accuracy para las configuraciones, y realizaremos una comparativa de todos ellos.

In [39]:
from itertools import product
from time import time

def getScores(estim,paramG,train_atts,train_label,test_atts,test_label):
    
    scores=[] #para almacenar los resultados de accuracy a devolver
    exTimes=[] #para almacenar los tiempos a devolver
    
    #Generamos las configuraciones de parametros como en GridS
    items = paramG.items()
    keys, values = zip(*items)
    v = list(product(*values))
    
    # Recorremos las configuraciones y almacenamos los resultados
    for param in v:
        
        params = dict(zip(keys,param))
        estim.set_params(**params)
        
        timeSum=0 #variable intermedia para mostrar la media de los tiempos
        
        for i in range(0,100):
            start = time()
            estim.fit(train_atts,train_label) #entrenamos con el conjunto de Train
            predictions = estim.predict(test_atts) #array de predicciones para el conjunto de Test
            end = time()
            timeSum += (end - start)
        exTimes.append(timeSum/100) #añadimos al array de tiempos la media de las 100 medidas de tiempo realizadas
        
        comparison = np.sum(predictions == test_label) #comparamos predicciones con test_label y sumamos los aciertos
        accuracy = comparison / len(predictions) #dividimos entre los casos totales
        scores.append(accuracy) #añadimos al array de scores el accuracy obtenido con la configuración de parámetros actual
        
    return (v,scores,exTimes) #devolvemos las configuraciones, los accuracys y los tiempos de cada configuracion

### 1.4.1. Estudio del algoritmo KNN

Con este clasificador, usaremos las combinaciones teniendo en cuenta el número de vecinos (k=1,5,10,50,100) y el método de evaluación de las distancias (uniforme o inversa de la distancia). Para ello usaremos nuestra función getScores.

#### 1.4.1.1. KNN con Pima

Lo primero de todo es imputar con la media los valores perdidos del dataset Pima.

In [40]:
imp=Imputer(missing_values='NaN', strategy="mean",axis=0)
#we fit it
impPima = imp.fit(train_attsPima)
# Finally we can use it to transform any dataframe:
train_attsPima_clean = impPima.transform(train_attsPima)
test_attsPima_clean = impPima.transform(test_attsPima)

Una vez hecho esto, podemos hacer una comparativa de los resultados obtenidos para todas las configuraciones del clasificador KNN.

In [41]:
paramsKNNPima,scoresKNNPima,exTimesKNNPima=getScores(neighbors.KNeighborsClassifier(),
                                                     { 'weights': ['uniform','distance'], 'n_neighbors' : [1,5,10,50,100] },
                                                     train_attsPima_clean,train_labelPima,
                                                     test_attsPima_clean,test_labelPima)
resultadosKNNPima=pd.DataFrame(list(zip(paramsKNNPima,scoresKNNPima,exTimesKNNPima)))
resultadosKNNPima.columns=['paramsKNNPima','scoresKNNPima','exTimesKNNPima']
resultadosKNNPima

,paramsKNNPima,scoresKNNPima,exTimesKNNPima
0,"(uniform, 1)",0.629870,0.002003
1,"(uniform, 5)",0.688312,0.002124
2,"(uniform, 10)",0.694805,0.002428
3,"(uniform, 50)",0.733766,0.003427
4,"(uniform, 100)",0.707792,0.004955
5,"(distance, 1)",0.629870,0.001996
6,"(distance, 5)",0.694805,0.002142
7,"(distance, 10)",0.701299,0.002342
8,"(distance, 50)",0.746753,0.003641
9,"(distance, 100)",0.727273,0.005281


Como podemos comprobar, la configuración con mejor accuracy sería usar k=50 y la inversa de la distancia (fila 8 de la tabla). Por tanto, si aumentamos K, podríamos pensar que se mejora el accuracy, pero solo hasta cierto punto, porque con K=100 (fila 9) ya obtenemos un accuracy peor. De forma general, usar un K muy alto se asemeja a un clasificador ZeroR, ya que ambos utilizarían la estrategia de clasificación por la clase mayoritaria.

Además, debemos tener en mente que cuanto mayor sea el parámetro K, mayor será el tiempo de ejecución. En este caso, el tiempo de ejecución se corresponde con el tiempo de realizar la predicción con el conjunto de Test. Esto es debido a que el entrenamiento en el clasificador KNN es muy rápido, ya que solo consiste en copiar la base de datos de los casos de Train.

#### 1.4.1.2. KNN con Wisconsin

Ahora estudiaremos las configuraciones obtenidas con el dataset Wisconsin. Primero debemos imputar los valores perdidos.

In [42]:
#we fit it
impWisc = imp.fit(train_attsWisc2)
# Finally we can use it to transform any dataframe:
train_attsWisc2_clean = impWisc.transform(train_attsWisc2)
test_attsWisc2_clean = impWisc.transform(test_attsWisc2)

Una vez hecho esto, realizamos la comparativa de los resultados obtenidos para todas las configuraciones del clasificador KNN.

In [43]:
paramsKNNWisc,scoresKNNWisc,exTimesKNNWisc=getScores(neighbors.KNeighborsClassifier(),
                                                     { 'weights': ['uniform','distance'], 'n_neighbors' : [1,5,10,50,100] },
                                                     train_attsWisc2_clean,train_labelWisc,
                                                     test_attsWisc2_clean,test_labelWisc)
resultadosKNNWisc=pd.DataFrame(list(zip(paramsKNNWisc,scoresKNNWisc,exTimesKNNWisc)))
resultadosKNNWisc.columns=['paramsKNNWisc','scoresKNNWisc','exTimesKNNWisc']
resultadosKNNWisc

,paramsKNNWisc,scoresKNNWisc,exTimesKNNWisc
0,"(uniform, 1)",0.964286,0.002401
1,"(uniform, 5)",0.964286,0.002179
2,"(uniform, 10)",0.964286,0.002381
3,"(uniform, 50)",0.971429,0.003233
4,"(uniform, 100)",0.964286,0.004371
5,"(distance, 1)",0.964286,0.002104
6,"(distance, 5)",0.964286,0.002229
7,"(distance, 10)",0.978571,0.002346
8,"(distance, 50)",0.971429,0.003307
9,"(distance, 100)",0.964286,0.004571


Como podemos comprobar, la configuración con mejor accuracy sería usar k=10 y la inversa de la distancia (fila 7 de la tabla). 

Con este dataset, a diferencia de lo que observamos con Pima, vemos que la mejor opción es usar valores de K algo más pequeños, ya que los accuracys empeoran al usar K's más grandes. Además, cuanto mayor es K, mayor será el tiempo de ejecución del algoritmo. 

Como ya mencionamos anteriormente, de forma general podemos decir que usar un K muy alto en KNN se asemeja a un clasificador ZeroR, ya que ambos utilizarían la estrategia de clasificación por la clase mayoritaria.

### 1.4.2. Estudio del algoritmo DecisionTree

Con este clasificador, realizaremos las combinaciones teniendo en cuenta el criterio (entropía o gini), la profundidad máxima del árbol, y el número mínimo de ejemplos por hoja. 

Para analizar la estructura de los árboles generados con las diferentes combinaciones de parámetros, vamos a usar (además de nuestra función getScores) la función getTreeNodes, que nos devuelve el número de nodos que presenta un árbol.

In [44]:
def getTreeNodes(estim,paramG,train_atts,train_label):
    
    nodes=[] #para almacenar el numero de nodos de cada arbol generado
    
    #Generamos las configuraciones de parametros como en GridS
    items = paramG.items()
    keys, values = zip(*items)
    v = list(product(*values))
    
    # Recorremos las configuraciones y almacenamos los resultados
    for param in v:
        
        params = dict(zip(keys,param))
        estim.set_params(**params)

        estim.fit(train_atts,train_label) #entrenamos con el conjunto de Train
        
        nodes.append(estim.tree_.node_count) #añadimos al array el numero de nodos del arbol actual
        
    return (v,nodes) #devolvemos las configuraciones de los arboles y el numero de nodos de cada uno de ellos

#### 1.4.2.1. Árbol con Pima

Comenzaremos con el dataset Pima. Vamos a hacer una comparativa de los resultados obtenidos para todas las configuraciones del árbol de clasificación, junto con la complejidad de cada árbol generado (representada por el número de nodos de cada árbol).

In [45]:
paramsTreePima,scoresTreePima,exTimesTreePima=getScores(tree.DecisionTreeClassifier(random_state=seed),
                                                        {'criterion': ["entropy","gini"],
                                                         'max_depth': [3, 5, 10], 
                                                         'min_samples_leaf': [3,5,10]},
                                                        train_attsPima_clean,train_labelPima,
                                                        test_attsPima_clean,test_labelPima)
resultadosTreePima=pd.DataFrame(list(zip(paramsTreePima,scoresTreePima,exTimesTreePima)))
resultadosTreePima.columns=['paramsTreePima','scoresTreePima','exTimesTreePima']
#resultadosTreePima

In [46]:
paramsTreePima,nodesTreePima=getTreeNodes(tree.DecisionTreeClassifier(random_state=seed),
                                          {'criterion': ["entropy","gini"],
                                           'max_depth': [3, 5, 10], 
                                           'min_samples_leaf': [3,5,10]},
                                          train_attsPima_clean,train_labelPima)
resultadosTreePima.assign(nodesTreePima = nodesTreePima)
#resultadosTreePima['nodesTreePima']=nodesTreePima

,paramsTreePima,scoresTreePima,exTimesTreePima,nodesTreePima
0,"(entropy, 3, 3)",0.746753,0.002374,15
1,"(entropy, 3, 5)",0.746753,0.002088,15
2,"(entropy, 3, 10)",0.746753,0.002269,15
3,"(entropy, 5, 3)",0.720779,0.003040,39
4,"(entropy, 5, 5)",0.720779,0.003044,41
5,"(entropy, 5, 10)",0.707792,0.002818,43
6,"(entropy, 10, 3)",0.727273,0.004216,131
7,"(entropy, 10, 5)",0.720779,0.003831,111
8,"(entropy, 10, 10)",0.727273,0.003169,73
9,"(gini, 3, 3)",0.753247,0.001406,15


Como vemos en la tabla, la configuración con mayor accuracy sería la que se encuentra en la fila 16: criterio gini, máxima profundidad del árbol igual a 10 y mínimo número de ejemplos por hoja igual a 5.

Sin embargo, esta configuración genera un árbol muy complejo, con más de 100 nodos. Por tanto, podríamos compararlo con el segundo mejor árbol generado (árbol de la fila 9 de la tabla), el cual presenta un accuracy algo más bajo, pero está compuesto por tan solo 15 nodos. Este caso necesitaría una profundidad máxima del árbol igual a 3, y un número mínimo de ejemplos por hoja igual a 3, para conseguir (casi) igualar el accuracy al del árbol de la fila 16, por lo que sería interesante seleccionar el árbol más simple para nuestra clasificación.

De esta forma podemos ver que debemos tener en cuenta otros factores además del accuracy para poder evaluar la efectividad del clasificador.

#### 1.4.2.2. Árbol con Wisconsin

Ahora haremos lo mismo con los árboles generados para el dataset Wisconsin.

In [47]:
paramsTreeWisc,scoresTreeWisc,exTimesTreeWisc=getScores(tree.DecisionTreeClassifier(random_state=seed),
                                                        {'criterion': ["entropy","gini"],
                                                         'max_depth': [3, 5, 10], 
                                                         'min_samples_leaf': [3,5,10]},
                                                        train_attsWisc2_clean,train_labelWisc,
                                                        test_attsWisc2_clean,test_labelWisc)
resultadosTreeWisc=pd.DataFrame(list(zip(paramsTreeWisc,scoresTreeWisc,exTimesTreeWisc)))
resultadosTreeWisc.columns=['paramsTreeWisc','scoresTreeWisc','exTimesTreeWisc']

In [48]:
paramsTreeWisc,nodesTreeWisc=getTreeNodes(tree.DecisionTreeClassifier(random_state=seed),
                                          {'criterion': ["entropy","gini"],
                                           'max_depth': [3, 5, 10], 
                                           'min_samples_leaf': [3,5,10]},
                                          train_attsWisc2_clean,train_labelWisc)
resultadosTreeWisc.assign(nodesTreeWisc = nodesTreeWisc)

,paramsTreeWisc,scoresTreeWisc,exTimesTreeWisc,nodesTreeWisc
0,"(entropy, 3, 3)",0.950000,0.001479,15
1,"(entropy, 3, 5)",0.950000,0.001279,15
2,"(entropy, 3, 10)",0.950000,0.000999,15
3,"(entropy, 5, 3)",0.950000,0.001119,35
4,"(entropy, 5, 5)",0.964286,0.001070,31
5,"(entropy, 5, 10)",0.950000,0.001018,25
6,"(entropy, 10, 3)",0.964286,0.001103,41
7,"(entropy, 10, 5)",0.964286,0.001070,33
8,"(entropy, 10, 10)",0.950000,0.001013,25
9,"(gini, 3, 3)",0.964286,0.000933,15


Como vemos en la tabla, una de las configuraciones con mayor accuracy sería la que se encuentra en la fila 4: criterio entropía, máxima profundidad del árbol igual a 5 y mínimo número de ejemplos por hoja igual a 5.

En este caso, los árboles no son demasiado complejos, todos tienen un número de nodos similar. Aún así podemos observar algunas pequeñas diferencias en dicho número.

Una configuración subóptima que sería interesante analizar es la que se encuentra en la fila 9 (criterio gini, máxima profundidad de 3 y mínimo número de ejemplos por hoja igual a 3), ya que obtiene un accuracy igual al árbol de la fila 4, pero presenta un número de nodos menor (se reduce a la mitad). Por ello, podría resultar interesante seleccionar el árbol más simple para nuestra clasificación.

De esta forma, al igual que en el dataset Pima, llegamos a la conclusión de que debemos tener en cuenta otros factores además del accuracy para poder evaluar la efectividad del clasificador.

### 1.4.3. Conclusiones del estudio de los algoritmos

Tras el estudio de los algoritmos, nos hemos dado cuenta de que dependiendo del problema y del dataset al que nos enfrentemos, es posible que la configuración de parámetros óptima para un determinado algoritmo no siempre sea la misma. 
* En el KNN con el dataset Pima hemos visto que es bueno aumentar la vecindad (hasta cierto punto). Sin embargo, con el dataset Wisconsin los resultados empeoran al usar vecindades grandes, y además se consumía demasiado tiempo.
* Con el árbol de decisión, hemos aprendido que es una buena práctica observar las configuraciones subóptimas del clasificador para obtener árboles más sencillos, en los cuales el tiempo de ejecución es menor y obtienen rendimientos similares a árboles más grandes.

# 2. Implementación de GridSearch manualmente

En este apartado realizamos una implementación manual de un GridSearh básico, con su constructor, y las funciones fit y predict, que son las que utilizamos en esta práctica.

Nuestra clase GridS tiene como atributos:
* estim: estimador o clasificador a evaluar,
* crossval: cómo realizar el proceso de validación cruzada durante la evaluación,
* score: tipo de métrica a utilizar para la evaluación de la configuración óptima,
* keys: nombre de los parámetros del clasificador a tener en cuenta en la evaluación,
* v: lista con todas las posibles combinaciones de parámetros del clasificador,
* bestScore: mejor puntuación obtenida con la configuración de parámetros del clasificador,
* bestParams: mejor configuración de parámetros del clasificador.

La función fit se encarga de realizar la validación cruzada para cada una de las diferentes combinaciones de parámetros contenidas en self.v, obteniendo así aquella configuración que ofrezca una mayor puntuación. Una vez obtenida, entrena el clasificador con el conjunto de datos que se le pasa como parámetro a la función fit.

La función predict simplemente hace la validación del clasificador con el conjunto que reciba como parámetro.

In [49]:
from itertools import product
from sklearn.model_selection import cross_val_score,StratifiedKFold

class GridS(object):
    def __init__(self,estim, paramG, score, crossval):
        items = paramG.items()
        self.keys, values = zip(*items)
        self.v = list(product(*values))
        self.estim=estim
        self.crossval=crossval
        self.score=score
        self.bestParams = []
            
    def fit(self,atts,label):
        self.bestScore=0
        for param in self.v:
            params = dict(zip(self.keys,param))
            self.estim.set_params(**params)
            scores = cross_val_score(estimator=self.estim, 
                                     X=atts, y=label, 
                                     scoring=self.score, 
                                     cv=self.crossval)
            if self.bestScore < scores.mean():
                self.bestScore=scores.mean()
                self.bestParams=params
        self.estim.set_params(**self.bestParams)
        self.estim.fit(atts,label)
        return self
    
    def predict(self, atts):
        return self.estim.predict(atts)

## 2.1. KNN con GridSearch manual

Creamos un Pipeline que tiene como primer paso el Imputer de los datos, y como segundo paso el clasificador KNN.

In [50]:
estimatorGSKNN = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("myKNN", neighbors.KNeighborsClassifier())])

Al igual que hicimos anteriormente con el GridSearch de scikit, creamos un objeto de la clase GridS. Le pasamos nuestro pipeline, y le indicamos que obtenga la mejor configuración de parámetros atendiendo al número de vecinos (de 1 a 5 vecinos), y el tipo de métrica a utilizar para valorar la distancia (uniforme o inversa de la distancia). El tipo de métrica a utilizar será de nuevo el accuracy, y utilizaremos un proceso de validación cruzada estratificada de 10 folds, con nuestra semilla. 

In [51]:
gsKNN = GridS(
    estim = estimatorGSKNN,
    paramG = 
        { 'myKNN__n_neighbors' : [1,2,3,4,5], 'myKNN__weights': ['uniform','distance'] },
    score = 'accuracy',
    crossval = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
)

### 2.1.1. Pima

A continuación utilizamos nuestro GridS en el dataset Pima y evaluamos los resultados. Como vemos, son los mismos que obtuvimos usando el GridSearch de scikit.

In [52]:
clsKNNPimaGS = gsKNN.fit(train_attsPima, train_labelPima)
predictionKNNPimaGS = gsKNN.predict(test_attsPima)
print('Accuracy Pima KNN + GridS:')
metrics.accuracy_score(test_labelPima, predictionKNNPimaGS)

Accuracy Pima KNN + GridS:


0.68831168831168832

In [53]:
print('Mejor configuración de parámetros (Pima KNN + GridS):')
clsKNNPimaGS.bestParams

Mejor configuración de parámetros (Pima KNN + GridS):


{'myKNN__n_neighbors': 5, 'myKNN__weights': 'uniform'}

In [54]:
# Precision
print('Precision Pima KNN + GridS:')
metrics.precision_score(test_labelPima, predictionKNNPimaGS, pos_label="tested_positive")

Precision Pima KNN + GridS:


0.5714285714285714

In [55]:
# Recall
print('Recall Pima KNN + GridS:')
metrics.recall_score(test_labelPima, predictionKNNPimaGS, pos_label="tested_positive")

Recall Pima KNN + GridS:


0.44444444444444442

### 2.1.2. Wisconsin

Hacemos lo mismo con el dataset Wisconsin. Como vemos, se obtienen los mismos resultados que teníamos usando el GridSearch de scikit.

In [56]:
clsKNNWiscGS = gsKNN.fit(train_attsWisc2, train_labelWisc)
predictionKNNWiscGS = gsKNN.predict(test_attsWisc2)
print('Accuracy Wiscconsin KNN + GridS:')
metrics.accuracy_score(test_labelWisc, predictionKNNWiscGS)

Accuracy Wiscconsin KNN + GridS:


0.97857142857142854

In [57]:
print('Mejor configuración de parámetros (Wisconsin KNN + GridS):')
clsKNNWiscGS.bestParams

Mejor configuración de parámetros (Wisconsin KNN + GridS):


{'myKNN__n_neighbors': 3, 'myKNN__weights': 'uniform'}

In [58]:
# Precision
print('Precision Wisconsin KNN + GridS:')
metrics.precision_score(test_labelWisc, predictionKNNWiscGS, pos_label="malignant")

Precision Wisconsin KNN + GridS:


0.97872340425531912

In [59]:
# Recall
print('Recall Wisconsin KNN + GridS:')
metrics.recall_score(test_labelWisc, predictionKNNWiscGS, pos_label="malignant")

Recall Wisconsin KNN + GridS:


0.95833333333333337

## 2.2. Árbol de decisión con GridSearch manual

Al igual que hicimos anteriormente con el GridSearch de scikit, creamos un objeto de la clase GridS. Le pasamos un clasificador DecisionTree, y le indicamos que obtenga la mejor configuración de parámetros atendiendo al criterio de evaluación (entropía o gini), la máxima profundidad del árbol, y el número mínimo de ejemplos por hoja. El tipo de métrica a utilizar será de nuevo el accuracy, y utilizaremos un proceso de validación cruzada estratificada de 10 folds, con nuestra semilla.

In [60]:
estimatorGSTree = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("myTree", tree.DecisionTreeClassifier(random_state=seed),)])

In [61]:
gsTree=GridS(
    estim = estimatorGSTree,
    paramG=
        {'myTree__criterion': ["entropy","gini"],'myTree__max_depth': [3, 5, 10, None], 'myTree__min_samples_leaf': [3,5,10]},
    score='accuracy',
    crossval=StratifiedKFold(n_splits=10, shuffle=False, random_state=seed)
)

### 2.2.1. Pima

A continuación utilizamos nuestro GridS en el dataset Pima y evaluamos los resultados. Como vemos, son los mismos que obtuvimos usando el GridSearch de scikit.

In [62]:
# We can fit and use the pipeline as usual
clsTreePimaGS = gsTree.fit(train_attsPima, train_labelPima)
predictionTreePimaGS = gsTree.predict(test_attsPima)
print('Accuracy Pima Tree + GridS:')
metrics.accuracy_score(test_labelPima, predictionTreePimaGS)

Accuracy Pima Tree + GridS:


0.72727272727272729

In [63]:
print('Mejor configuración de parámetros (Pima Tree + GridS):')
clsTreePimaGS.bestParams

Mejor configuración de parámetros (Pima Tree + GridS):


{'myTree__criterion': 'entropy',
 'myTree__max_depth': 10,
 'myTree__min_samples_leaf': 10}

In [64]:
# Precision
print('Precision Pima Tree + GridS:')
metrics.precision_score(test_labelPima, predictionTreePimaGS, pos_label="tested_positive")

Precision Pima Tree + GridS:


0.61111111111111116

In [65]:
# Recall
print('Recall Pima Tree + GridS:')
metrics.recall_score(test_labelPima, predictionTreePimaGS, pos_label="tested_positive")

Recall Pima Tree + GridS:


0.61111111111111116

### 2.2.2. Wisconsin

Hacemos lo mismo con el dataset Wisconsin. Como vemos, se obtienen los mismos resultados que teníamos usando el GridSearch de scikit.

In [66]:
# We can fit and use the pipeline as usual
clsTreeWiscGS = gsTree.fit(train_attsWisc2, train_labelWisc)
predictionTreeWiscGS = gsTree.predict(test_attsWisc2)
print('Accuracy Wisconsin Tree + GridS:')
metrics.accuracy_score(test_labelWisc, predictionTreeWiscGS)

Accuracy Wisconsin Tree + GridS:


0.95714285714285718

In [67]:
print('Mejor configuración de parámetros (Wisconsin Tree + GridS):')
clsTreeWiscGS.bestParams

Mejor configuración de parámetros (Wisconsin Tree + GridS):


{'myTree__criterion': 'gini',
 'myTree__max_depth': 5,
 'myTree__min_samples_leaf': 5}

In [68]:
# Precision
print('Precision Wisconsin Tree + GridS:')
metrics.precision_score(test_labelWisc, predictionTreeWiscGS, pos_label="malignant")

Precision Wisconsin Tree + GridS:


0.95652173913043481

In [69]:
# Recall
print('Recall Wisconsin Tree + GridS:')
metrics.recall_score(test_labelWisc, predictionTreeWiscGS, pos_label="malignant")

Recall Wisconsin Tree + GridS:


0.91666666666666663

# 3. Estudio de un kernel de kaggle

https://www.kaggle.com/kanncaa1/roc-curve-with-k-fold-cv

# 4. Extras: Algoritmo RandomizedSearchCV

In [70]:
from sklearn.model_selection import RandomizedSearchCV

En esta sección realizaremos experimentos utilizando el algoritmo RandomizedSearch de la librería scikit-learn para descubrir las configuraciones óptimas de los clasificadores DecisionTree y KNN para los datasets Pima y Wisconsin. Seguiremos el mismo proceso que hicimos con GridSearch.

## 4.1. Árbol de decisión con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

### 4.1.1. Pipeline

Primero creamos un Pipeline con dos pasos: el primero es imputar los valores perdidos siguiendo la estrategia de rellenar con la media, y el segundo paso es un árbol.

In [71]:
# We define the pipeline as a set of tuples
estimatorTreeRS = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("RSTree", tree.DecisionTreeClassifier(random_state=seed))])

### 4.1.2. RandomizedSearch

Aquí definimos nuestro RandomizedSearch con los parámetros elegidos, usando nuestro Pipeline.

En nuestro caso hemos elegido los parámetros 'criterion' (puede ser "gini", que usa la impureza Gini como función, o “entropy”, usando la ganancia de información), 'max_depth' (para indicar la máxima profundidad del árbol), y 'min_samples_leaf' (para indicar el número mínimo de ejemplos que queremos que haya en cada hoja del árbol).

In [72]:
clfTreeRS = RandomizedSearchCV(
    estimator = estimatorTreeRS,
    param_distributions =
        {'RSTree__criterion': ["entropy","gini"],'RSTree__max_depth': [3, 5, 10, None], 'RSTree__min_samples_leaf': [3,5,10]},
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False,
    random_state=seed
)

### 4.1.3. Pima

Después de crear el Pipeline, lo ejecutamos con el dataset Pima para entrenar el clasificador con el conjunto de Train, validamos con nuestro conjunto de Test, y después imprimimos el accuracy obtenido con la mejor configuración de parámetros del árbol encontrada por nuestro RandomizedSearch.

In [73]:
# We can fit and use the pipeline as usual
clsTreePimaRS = clfTreeRS.fit(train_attsPima, train_labelPima)
predictionTreePimaRS = clfTreeRS.predict(test_attsPima)
print('Accuracy Pima Tree with RS:')
metrics.accuracy_score(test_labelPima, predictionTreePima)

Accuracy Pima Tree with RS:


0.72727272727272729

Por último, nos parece interesante mostrar la mejor configuración de los parámetros del árbol que ha encontrado el RandomizedSearch. En este caso, la mejor configuración es utilizar el criterio de entropía para evaluar las variables del árbol, emplear una profundidad máxima del árbol de 3, y un número mínimo de ejemplos por hoja igual a 10.

In [74]:
print('Mejor configuración de parámetros (Pima Tree) con RS:')
clsTreePimaRS.best_params_

Mejor configuración de parámetros (Pima Tree) con RS:


{'RSTree__criterion': 'entropy',
 'RSTree__max_depth': 3,
 'RSTree__min_samples_leaf': 10}

Además del accuracy, también es interesante mostrar la matriz de confusión obtenida, y las medidas de precision y recall.

In [75]:
print('Confusion matrix Pima Tree with RS:')
metrics.confusion_matrix(test_labelPima, predictionTreePimaRS)

Confusion matrix Pima Tree with RS:


array([[85, 15],
       [24, 30]])

In [76]:
# Recall
print('Recall Pima Tree with RS:')
metrics.recall_score(test_labelPima, predictionTreePimaRS, pos_label="tested_positive")

Recall Pima Tree with RS:


0.55555555555555558

In [77]:
# Precision
print('Precision Pima Tree with RS:')
metrics.precision_score(test_labelPima, predictionTreePimaRS, pos_label="tested_positive")

Precision Pima Tree with RS:


0.66666666666666663

### 4.1.4. Wisconsin

Ahora realizamos el mismo procedimiento con el dataset Wisconsin. Usando el Pipeline creado en el apartado 4.1.2., entrenamos, validamos y obtenemos las métricas.

In [78]:
# We can fit and use the pipeline as usual
clsTreeWiscRS = clfTreeRS.fit(train_attsWisc2, train_labelWisc)
predictionTreeWiscRS = clfTreeRS.predict(test_attsWisc2)
print('Accuracy Wisconsin Tree with RS:')
metrics.accuracy_score(test_labelWisc, predictionTreeWiscRS)


Accuracy Wisconsin Tree with RS:


0.95714285714285718

Ahora mostramos la mejor configuración obtenida con el RandomizedSearch, que en este caso consiste en usar el criterio gini para el árbol, una profundidad máxima de 5, y un número mínimo de ejemplos por hoja igual a 3.

In [79]:
print('Mejor configuración de parámetros (Wisconsin Tree) con RS:')
clsTreeWiscRS.best_params_

Mejor configuración de parámetros (Wisconsin Tree) con RS:


{'RSTree__criterion': 'gini',
 'RSTree__max_depth': 10,
 'RSTree__min_samples_leaf': 5}

A continuación, mostramos la matriz de confusión, precision y recall, igual que hicimos con el dataset Pima.

In [80]:
print('Confusion matrix Wisconsin Tree with RS:')
metrics.confusion_matrix(test_labelWisc, predictionTreeWiscRS)

Confusion matrix Wisconsin Tree with RS:


array([[90,  2],
       [ 4, 44]])

In [81]:
# Recall
print('Recall Wisconsin Tree with RS:')
metrics.recall_score(test_labelWisc, predictionTreeWiscRS, pos_label="malignant")

Recall Wisconsin Tree with RS:


0.91666666666666663

In [82]:
# Precision
print('Precision Wisconsin Tree with RS:')
metrics.precision_score(test_labelWisc, predictionTreeWiscRS, pos_label="malignant")

Precision Wisconsin Tree with RS:


0.95652173913043481

## 4.2. KNN con preprocesamiento de los datos durante la validación cruzada (transformers y pipelines)

En esta parte vamos a usar un KNN como clasificador para nuestro Pipeline.

### 4.2.1. Pipeline

Creamos el Pipeline con el imputer como primer paso, y como segundo paso, el clasificador KNN.

In [83]:
# We define the pipeline as a set of tuples
estimatorKNNRS = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("RSKNN", neighbors.KNeighborsClassifier())])

### 4.2.2. RandomizedSearch

Primero definimos los parámetros del algoritmo de KNN que queremos que el RandomizedSearch tenga en cuenta a la hora de buscar la configuración óptima. En el algoritmo KNeighborsClassifier no se nos permite elegir un random_state como en el caso del árbol para poder hacer reproducibles nuestros experimentos, por lo que solo usaremos la semilla en el proceso de validación cruzada que se realiza en el RandomizedSearch.

En nuestro caso, para el KNN hemos añadido el número de vecinos, y el tipo de métrica a utilizar para valorar las distancias de los vecinos (puede ser la distancia o la inversa de la distancia).

In [84]:
clfKNNRS = RandomizedSearchCV(
    estimator = estimatorKNNRS,
    param_distributions = 
        { 'RSKNN__n_neighbors' : [1,2,3,4,5], 'RSKNN__weights': ['uniform','distance'] },
    scoring = 'accuracy',
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=seed), 
    iid=False,
    random_state=seed
)

### 4.2.3. Pima

Primero entrenamos, validamos y obtenemos el accuracy en los datos del dataset Pima.

In [85]:
# We can fit and use the pipeline as usual
clsKNNPimaRS = clfKNNRS.fit(train_attsPima, train_labelPima)
predictionKNNPimaRS = clfKNNRS.predict(test_attsPima)
print('Accuracy Pima KNN with RS:')
metrics.accuracy_score(test_labelPima, predictionKNNPimaRS)

Accuracy Pima KNN with RS:


0.68831168831168832

Comprobamos cuál ha sido la mejor configuración de parámetros obtenida por el RandomizedSearch. En este caso es utilizar 5 vecinos, usando la métrica uniforme para medir las distancias.

In [86]:
print('Mejor configuración de parámetros (Pima KNN) con RS:')
clsKNNPimaRS.best_params_

Mejor configuración de parámetros (Pima KNN) con RS:


{'RSKNN__n_neighbors': 5, 'RSKNN__weights': 'uniform'}

A continuación mostramos la matriz de confusión, precision y recall obtenidos.

In [87]:
print('Confusion matrix Pima KNN with RS:')
metrics.confusion_matrix(test_labelPima, predictionKNNPimaRS)

Confusion matrix Pima KNN with RS:


array([[82, 18],
       [30, 24]])

In [88]:
# Recall
print('Recall Pima KNN with RS:')
metrics.recall_score(test_labelPima, predictionKNNPimaRS, pos_label="tested_positive")

Recall Pima KNN with RS:


0.44444444444444442

In [89]:
# Precision
print('Precision Pima KNN with RS:')
metrics.precision_score(test_labelPima, predictionKNNPimaRS, pos_label="tested_positive")

Precision Pima KNN with RS:


0.5714285714285714

### 4.2.4. Wisconsin

Con los datos del dataset Wisconsin, entrenamos nuestro clasificador, validamos y obtenemos el accuracy.

In [90]:
# We can fit and use the pipeline as usual
clsKNNWiscRS = clfKNNRS.fit(train_attsWisc2, train_labelWisc)
predictionKNNWiscRS = clfKNNRS.predict(test_attsWisc2)
print('Accuracy Wisconsin KNN with RS:')
metrics.accuracy_score(test_labelWisc, predictionKNNWiscRS)

Accuracy Wisconsin KNN with RS:


0.97857142857142854

Los mejores parámetros obtenidos por el RandomizedSearch para el KNN en este dataset son utilizar 3 vecinos, usando la métrica uniforme para medir las distancias.

In [91]:
print('Mejor configuración de parámetros (Wisconsin KNN) con RS:')
clsKNNWiscRS.best_params_

Mejor configuración de parámetros (Wisconsin KNN) con RS:


{'RSKNN__n_neighbors': 3, 'RSKNN__weights': 'uniform'}

A continuación, mostramos la matriz de confusión, precision y recall obtenidos.

In [92]:
print('Confusion matrix Wisconsin KNN with RS:')
metrics.confusion_matrix(test_labelWisc, predictionKNNWiscRS)

Confusion matrix Wisconsin KNN with RS:


array([[91,  1],
       [ 2, 46]])

In [93]:
# Recall
print('Recall Wisconsin KNN with RS:')
metrics.recall_score(test_labelWisc, predictionKNNWiscRS, pos_label="malignant")

Recall Wisconsin KNN with RS:


0.95833333333333337

In [94]:
# Precision
print('Precision Wisconsin KNN with RS:')
metrics.precision_score(test_labelWisc, predictionKNNWiscRS, pos_label="malignant")

Precision Wisconsin KNN with RS:


0.97872340425531912

Como podemos observar, los resultados usando RandomizedSearch y GridSearch son muy similares.